In [9]:
import os
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [10]:
# =============================
# 🚀 CONFIGURATIONS
# =============================
DATASET_DIR = "../FBMM/Unsplitted_Ready_Sets/set_01_class_balanced_augs_applied_splitted"
MODEL_SAVE_PATH = "./models/efficientnet_b0_emotion.h5"
BATCH_SIZE = 128  # Increased batch size for better GPU utilization
IMG_SIZE = (260, 260)
EPOCHS = 50
INITIAL_LR = 0.001

In [11]:
# =============================
# 🚀 GPU CONFIGURATION (Enable Memory Growth)
# =============================
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        pass  # No logging

In [12]:
# =============================
# 🚀 OPTIMIZED DATA LOADING (TF.DATA.DATASET)
# =============================
def parse_image(filename, label):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE) / 255.0  # Normalize
    return img, label

def load_dataset(data_dir, batch_size):
    class_names = sorted(os.listdir(data_dir))
    class_indices = {cls: i for i, cls in enumerate(class_names)}

    filepaths, labels = [], []
    for cls in class_names:
        cls_path = os.path.join(data_dir, cls)
        for file in os.listdir(cls_path):
            filepaths.append(os.path.join(cls_path, file))
            labels.append(class_indices[cls])

    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))
    dataset = dataset.shuffle(len(filepaths))
    dataset = dataset.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset, len(class_names)

# 🚀 Load datasets
train_dataset, num_classes = load_dataset(os.path.join(DATASET_DIR, "train"), BATCH_SIZE)
val_dataset, _ = load_dataset(os.path.join(DATASET_DIR, "val"), BATCH_SIZE)
test_dataset, _ = load_dataset(os.path.join(DATASET_DIR, "test"), BATCH_SIZE)

In [13]:
# =============================
# 🚀 MODEL DEFINITION (Fine-Tuning)
# =============================
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(260, 260, 3))

# Unlock the last layer and 20% of other layers for training
fine_tune_at = int(len(base_model.layers) * 0.8)  # Unlock last 20% of layers
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
    layer.trainable = True

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
output_layer = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output_layer)

# 🚀 Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LR),
    loss="sparse_categorical_crossentropy",  # Use sparse labels (not one-hot)
    metrics=["accuracy"]
)

In [14]:
# =============================
# 🚀 CALLBACKS (Only Progress Bar)
# =============================
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3),
    ModelCheckpoint(MODEL_SAVE_PATH, monitor="val_accuracy", save_best_only=True),
]

In [15]:
# =============================
# 🚀 TRAIN THE MODEL (Only Progress Bar)
# =============================
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=callbacks,
    verbose=1  # ✅ Shows only the progress bar
)

# =============================
# 🚀 SAVE FINAL MODEL
# =============================
model.save(MODEL_SAVE_PATH)

Epoch 1/50
969/969 [==============================] - ETA: 0s - loss: 1.9491 - accuracy: 0.1421

TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.